In [1]:
import os
import requests
import json
from scripts.onto import FHIRTerminologyClient
from dotenv import load_dotenv

#### Concept retrieval from the London terminology server

Based on a example value set that was made on Snapper. The SNOMED-CT codes contained with the value set are parsed from the resulting json into a list. For general request API guidance, see https://developer.nhs.uk/apis/gpconnect-1-2-2/development_fhir_api_guidance.html#requests

In [3]:
# loads CLIENT_ID and CLIENT_SECRET that are set locally in .env, and initialises FHIR client

load_dotenv() 
fhir_client = FHIRTerminologyClient(endpoint_type='authoring') 

In [4]:
# retrieves list of concept codes given a value set id

value_set_id = '0b7fb784-d8b1-4a1a-9575-6fef1b53dc92'

fhir_client.retrieve_concept_codes_from_id(value_set_id)

['59621000', '1201005']

In [7]:
# retrieve list of concept codes using a value set url

url = 'http://aic/hypertensiondisordertest'

fhir_client.retrieve_concept_codes_from_url(url)

hypertensiondisordertest


['59621000', '1201005']

#### Experimenting with GET requests to API server

In [6]:
def get_access_token():
    """
    Given a CLIENT_ID and CLIENT_SECRET (set as environmental variables)
    Returns access token for session from OneLondon terminology server
    """
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')

    if not client_id or not client_secret:
        raise ValueError("Client ID and/or Client Secret not set in environment variables.")

    # endpoint url (see https://ontology.onelondon.online/)
    url = 'https://ontology.onelondon.online/authorisation/auth/realms/terminology/protocol/openid-connect/token'
    
    # define request contents
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret
    }
    
    # POST request
    try:
        response = requests.post(url, headers=headers, data=data)
    
        # check HTTP status code
        response.raise_for_status()
    
        # get access token
        access_token = response.json().get('access_token')
        if not access_token:
            raise ValueError("Failed to find access token.")

        return access_token
    
    except requests.RequestException as e:
        print(f"Unable to request: {e}")
        return None

In [7]:
# get access token

access_token = get_access_token()

In [8]:
# search for value sets by filtering on name

url = 'https://ontology.onelondon.online/authoring/fhir/ValueSet?name=Hypertension'

headers = {
    "Authorization": f"Bearer {access_token}"
}

response = requests.get(url, headers=headers)
response.json()

{'resourceType': 'Bundle',
 'id': 'fb3dc75d-460f-3731-b862-961d6094623a',
 'meta': {'lastUpdated': '2024-06-27T14:17:25.640+00:00',
  'tag': [{'system': 'http://terminology.hl7.org/CodeSystem/v3-ObservationValue',
    'code': 'SUBSETTED',
    'display': 'Resource encoded in summary mode'}]},
 'type': 'searchset',
 'total': 1,
 'link': [{'relation': 'self',
   'url': 'https://ontology.onelondon.online/authoring/fhir/ValueSet?_summary=true&name=Hypertension'}],
 'entry': [{'fullUrl': 'https://ontology.onelondon.online/authoring/fhir/ValueSet/0b7fb784-d8b1-4a1a-9575-6fef1b53dc92',
   'resource': {'resourceType': 'ValueSet',
    'id': '0b7fb784-d8b1-4a1a-9575-6fef1b53dc92',
    'meta': {'versionId': '1',
     'lastUpdated': '2024-06-20T10:45:52.407+00:00',
     'tag': [{'system': 'http://terminology.hl7.org/CodeSystem/v3-ObservationValue',
       'code': 'SUBSETTED',
       'display': 'Resource encoded in summary mode'}]},
    'url': 'http://aic/hypertensiondisordertest',
    'name': 'Hype

In [10]:
# retrieve a value set by requesting from [endpoint url]/[type]/[id]

url = 'https://ontology.onelondon.online/authoring/fhir/ValueSet/0b7fb784-d8b1-4a1a-9575-6fef1b53dc92'

headers = {
    "Authorization": f"Bearer {access_token}"
}

response = requests.get(url, headers=headers)
value_set = response.json()

display(value_set)

{'resourceType': 'ValueSet',
 'id': '0b7fb784-d8b1-4a1a-9575-6fef1b53dc92',
 'meta': {'versionId': '1', 'lastUpdated': '2024-06-20T10:45:52.407+00:00'},
 'text': {'status': 'generated',
  'div': '<div xmlns="http://www.w3.org/1999/xhtml"><h2>hypertensiondisordertest</h2><tt>http://aic/hypertensiondisordertest</tt><p>Test value set for SNOMED hypertension codes</p></div>'},
 'url': 'http://aic/hypertensiondisordertest',
 'name': 'Hypertensiondisordertest',
 'title': 'hypertensiondisordertest',
 'status': 'active',
 'experimental': True,
 'contact': [{'telecom': [{'system': 'email'}]}],
 'description': 'Test value set for SNOMED hypertension codes',
 'compose': {'include': [{'system': 'http://snomed.info/sct',
    'concept': [{'code': '59621000', 'display': 'Essential hypertension'},
     {'code': '1201005', 'display': 'Benign essential hypertension'}]}]}}

In [12]:
# retrieve a value set from canonical url

url = 'https://ontology.onelondon.online/authoring/fhir/ValueSet/$expand?url=http://aic/hypertensiondisordertest'

headers = {
    "Authorization": f"Bearer {access_token}"
}

response = requests.get(url, headers=headers)
value_set = response.json()
display(value_set)

{'resourceType': 'ValueSet',
 'url': 'http://aic/hypertensiondisordertest',
 'name': 'Hypertensiondisordertest',
 'title': 'hypertensiondisordertest',
 'status': 'active',
 'experimental': True,
 'expansion': {'identifier': 'urn:uuid:a5d84e9f-11bb-4a5e-9c87-a70af51178b2',
  'timestamp': '2024-06-27T14:18:30+00:00',
  'total': 2,
  'parameter': [{'name': 'version',
    'valueUri': 'http://snomed.info/sct|http://snomed.info/sct/83821000000107/version/20240605'},
   {'name': 'used-codesystem',
    'valueUri': 'http://snomed.info/sct|http://snomed.info/sct/83821000000107/version/20240605'},
   {'name': 'warning-experimental',
    'valueUri': 'http://aic/hypertensiondisordertest'}],
  'contains': [{'system': 'http://snomed.info/sct',
    'code': '59621000',
    'display': 'Essential hypertension'},
   {'system': 'http://snomed.info/sct',
    'code': '1201005',
    'display': 'Benign essential hypertension'}]}}

In [13]:
value_set.get('name')

'Hypertensiondisordertest'

In [14]:
value_set.get('expansion', {}).get('contains', [])

[{'system': 'http://snomed.info/sct',
  'code': '59621000',
  'display': 'Essential hypertension'},
 {'system': 'http://snomed.info/sct',
  'code': '1201005',
  'display': 'Benign essential hypertension'}]

In [15]:
code_list = [item['code'] for item in value_set.get('expansion', {}).get('contains', [])]
code_list

['59621000', '1201005']